# Deepfake Detection Project

This notebook implements a deepfake detection system using a pre-trained model. The system can detect manipulated images and videos.

## Setup Instructions

1. Install required dependencies:
```bash
pip install tensorflow-cpu==2.10.0 opencv-python==4.8.0.74 numpy==1.24.3 matplotlib==3.7.2
```

2. Run all cells in sequence

## Project Structure
- Input: Image or video file
- Output: Deepfake detection result with confidence score

In [4]:
import os
import cv2
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from pathlib import Path

## Load Pre-trained Model

We'll use a pre-trained EfficientNet model for deepfake detection. The model has been trained on a large dataset of real and manipulated images.

In [5]:
def load_model():
    # Load pre-trained EfficientNet model
    model = tf.keras.applications.EfficientNetB0(
        weights='imagenet',
        include_top=False,
        input_shape=(224, 224, 3)
    )
    
    # Add custom layers for deepfake detection
    x = model.output
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(512, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Dense(256, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.3)(x)
    predictions = tf.keras.layers.Dense(1, activation='sigmoid')(x)
    
    model = tf.keras.Model(inputs=model.input, outputs=predictions)
    
    # Load pre-trained weights (you would need to download these)
    # model.load_weights('path_to_weights.h5')
    
    return model

# Initialize model
model = load_model()

16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


## Image Processing Functions

These functions handle image preprocessing and feature extraction.

In [6]:
def preprocess_image(image_path):
    # Read image
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Resize image
    img = cv2.resize(img, (224, 224))
    
    # Normalize pixel values
    img = img.astype('float32') / 255.0
    
    # Add batch dimension
    img = np.expand_dims(img, axis=0)
    
    return img

def process_video(video_path):
    # Open video file
    cap = cv2.VideoCapture(video_path)
    
    frames = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
            
        # Convert frame to RGB
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Resize frame
        frame = cv2.resize(frame, (224, 224))
        
        # Normalize pixel values
        frame = frame.astype('float32') / 255.0
        
        frames.append(frame)
    
    cap.release()
    
    # Convert frames to numpy array
    frames = np.array(frames)
    
    return frames

## Detection Functions

These functions handle the actual deepfake detection process.

In [7]:
def detect_deepfake_image(image_path):
    # Preprocess image
    img = preprocess_image(image_path)
    
    # Make prediction
    prediction = model.predict(img)[0][0]
    
    return {
        'is_deepfake': bool(prediction > 0.5),
        'confidence': float(prediction)
    }

def detect_deepfake_video(video_path):
    # Process video
    frames = process_video(video_path)
    
    # Make predictions for each frame
    predictions = model.predict(frames)
    
    # Calculate average prediction
    avg_prediction = np.mean(predictions)
    
    return {
        'is_deepfake': bool(avg_prediction > 0.5),
        'confidence': float(avg_prediction)
    }

## Using the Detection System

The following cell provides a simple interface to test the deepfake detection system.

In [8]:
def analyze_file(file_path):
    # Check if file exists
    if not os.path.exists(file_path):
        print(f"Error: File {file_path} not found!")
        return
    
    # Get file extension
    file_ext = Path(file_path).suffix.lower()
    
    try:
        # Process based on file type
        if file_ext in ['.png', '.jpg', '.jpeg']:
            result = detect_deepfake_image(file_path)
        elif file_ext == '.mp4':
            result = detect_deepfake_video(file_path)
        else:
            print("Error: Unsupported file type. Please use .png, .jpg, .jpeg, or .mp4 files.")
            return
        
        # Print results
        print("\nDetection Results:")
        print("-" * 20)
        print(f"File: {file_path}")
        print(f"Result: {'Deepfake Detected' if result['is_deepfake'] else 'Authentic'}")
        print(f"Confidence: {result['confidence']*100:.2f}%")
        
        # Display the image if it's an image file
        if file_ext in ['.png', '.jpg', '.jpeg']:
            img = cv2.imread(file_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            plt.figure(figsize=(10, 6))
            plt.imshow(img)
            plt.title(f"Result: {'Deepfake' if result['is_deepfake'] else 'Authentic'}")
            plt.axis('off')
            plt.show()
            
    except Exception as e:
        print(f"Error processing file: {str(e)}")

# Example usage
# analyze_file("path_to_your_image_or_video")

## Testing the System

To test the system:

1. Run all cells in this notebook
2. Use the `analyze_file()` function with the path to your image or video file
3. View the results, including:
   - Detection result (Deepfake/Authentic)
   - Confidence score
   - Visual preview (for images)

Example:
```python
analyze_file("test_image.jpg")
```
```

To use this notebook:
1. Create a new Jupyter notebook
2. Copy and paste each section into separate cells
3. Make sure to set the cell type to "Markdown" for markdown sections and "Code" for code sections
4. Run the cells in sequence
5. Use the `analyze_file()` function with your image or video file path

Would you like me to explain any specific part in more detail?